In [1]:
from tavily import TavilyClient

import os
import pandas as pd
import nest_asyncio


In [ ]:
# This is a workaround for a bug in the nano_graphrag library
nest_asyncio.apply()

# Set the API key
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_API_KEY"

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


## Prepare data

In [5]:
import pandas as pd

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

filtered_data_train = []
filtered_data_test = []

for query in train_df["question"]:
    filtered_data_train.append({"Question": query})

for query in test_df["question"]:
    filtered_data_test.append({"Question": query})

filtered_data_train = filtered_data_train[:2]

In [ ]:
from script.prompt_manager import prepare_input_prompts
from script.initialize_tokenizer import initialize_tokenizer

dataset_name = "gpqa" # the prompt used for the GPQA dataset in the paper code because it's a dataset of multiple choice questions too.

tokenizer = initialize_tokenizer(model_name="gpt-4o")

# Prepare input prompts
input_list_train, active_sequences_train = prepare_input_prompts(
    filtered_data=filtered_data_train,
    model_path=None,
    dataset_name=dataset_name,
    tokenizer=tokenizer,
)

input_list_test, active_sequences_test = prepare_input_prompts(
    filtered_data=filtered_data_test,
    model_path=None,
    dataset_name=dataset_name,
    tokenizer=tokenizer,
)

/Users/yulin/anaconda3/envs/agentic_reasoning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Training data inference

In [7]:
from config import  END_SEARCH_QUERY, END_CODE_QUERY


model_args = {
    "stop_tokens": [END_SEARCH_QUERY, END_CODE_QUERY, tokenizer.eos_token],
    "model": "gpt-4o",
    "temperature": 0.3,
    "top_p": 0.95,
    "top_k": 50,
    "frequency_penalty": 0.3,
}



In [ ]:
from script.run_reasoning import run_reasoning 


accuracy_train = 0
final_results_train_complete = []
repetition = 5
for i in range(repetition):
    # Reset the finished flag for each sequence
    for j in range(len(active_sequences_train)):
        active_sequences_train[j]['finished'] = False 
    # Run reasoning
    sequences_train = run_reasoning(filtered_data_train, active_sequences_train, forcing_research=True, code_mode_name="gpt-4o", general_model_args=model_args, tokenizer=tokenizer, tavily_client=tavily_client)
    final_results_train = [active_sequences_train[i]['final_result'] for i in range(len(active_sequences_train))]
    print(f"Final results: {final_results_train}")
    
    accuracy_train += sum([1 for i in range(len(final_results_train)) if final_results_train[i] == train_df['answer'][i]])/len(final_results_train)
    final_results_train_complete.append(final_results_train)
    
accuracy_train /= repetition
print(f"Accuracy on training set: {accuracy_train}")
    


## Test data inference

In [ ]:
from script.run_reasoning import run_reasoning 


final_results_test_complete = []
repetition = 5
for i in range(repetition):
    for j in range(len(active_sequences_test)):
        active_sequences_test[j]['finished'] = False 
    sequences_test = run_reasoning(filtered_data_test, active_sequences_test, forcing_research=True, code_mode_name="gpt-4o", general_model_args=model_args, tokenizer=tokenizer, tavily_client=tavily_client)
    final_results_test = [active_sequences_test[i]['final_result'] for i in range(len(active_sequences_test))]
    print(f"Final results: {final_results_test}")
    final_results_test_complete.append(final_results_test)
    
    
print("final_results_test_complete", final_results_test_complete)

In [ ]:
import numpy as np
import csv

final_results_test_complete = np.array(final_results_test_complete).T.tolist()


def save_predictions_to_csv(predictions_list, filename):
    """Save formatted predictions to CSV."""
    header = ["question_id"] + [f"prediction_{i+1}" for i in range(5)]
    
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(header)  
        for i, predictions in enumerate(predictions_list, start=1):
            writer.writerow([i] + predictions)

# Save the formatted predictions
save_predictions_to_csv(final_results_test_complete, filename="./output/predictions_test.csv")
